In [ ]:
# Librerias basicas
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sns
import os
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

***

In [ ]:
# Importar los datos
dataset = pd.read_csv('Wine.csv')

# Division en features y target
X = dataset.iloc[:, 0:13].values
y = dataset.iloc[:, 13].values
features_numericas = dataset.iloc[:, 0:13]

***

# 1. EDA

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

 ## 1.1 Datos faltantes

In [ ]:
# Comprobar datos faltantes
dataset.isnull().sum()

Se comprueba que no existen datos faltantes

## 1.2 Cardinalidad del target

In [ ]:
# Comprobacion cardinalidad del target
target = dataset.Customer_Segment
target = target.to_frame(name='type')

In [ ]:
for col in target:
    print ('\nLa cantidad de observaciones del target expresado en valor numérico es la siguiente:')
    print (target[col].value_counts())
    print ('\nEl porcentaje de observaciones de cada categoría del target con respecto al total expresado en % es el siguiente:')
    print (round(target[col].value_counts() / len(target[col])*100 ))

In [ ]:
# Representacion grafica de la cardinalidad del target
fig = plt.figure(figsize=(10,5))
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="Customer_Segment", data=dataset)

## 1.3 Comprobar existencia de outliers

In [ ]:
# Comprobar existencia de outliers
# Cálculo de los valores inferiores , superiores y sus porcentajes, coeficiente de 1,5
# Cálculo de los valores extremos inferiores ,superiores y sus porcentajes, coeficiente de 3.
# Creación de dataframe con los resultados.

Columnas = []
LimiteInferior = []
LimiteSuperior = []
LimiteInferiorPor = []
LimiteSuperiorPorc = []
LimiteExInferior = []
LimiteExtSuperior = []
LimiteExtInferiorPor = []
LimiteExtSuperiorPorc = []

for (index, colname) in  enumerate(features_numericas):    
    
    IQR = features_numericas[colname].quantile(0.75) - features_numericas[colname].quantile(0.25)
    limSup = features_numericas[colname].quantile(0.75) + (IQR *  1.5)
    limInf = features_numericas[colname].quantile(0.25) - (IQR *  1.5)
    limSupExt = features_numericas[colname].quantile(0.75) + (IQR *  3)
    limInfExt = features_numericas[colname].quantile(0.25) - (IQR *  3)
    total = features_numericas[colname].shape[0]
    limSupPorc = (round(features_numericas[features_numericas[colname] > limSup].shape[0]/total,2))*100
    limInfPorc = (round(features_numericas[features_numericas[colname] < limInf].shape[0]/total,2))*100
    limSupExtPorc = (round(features_numericas[features_numericas[colname] > limSupExt].shape[0]/total,2))*100
    limInfExtPorc = (round(features_numericas[features_numericas[colname] < limInfExt].shape[0]/total,2))*100
    
    Columnas.append(colname)
    LimiteInferior.append(limInf)
    LimiteSuperior.append(limSup)
    LimiteInferiorPor.append(limInfPorc)
    LimiteSuperiorPorc.append(limSupPorc)
    LimiteExInferior.append(limInfExt)
    LimiteExtSuperior.append(limSupExt)
    LimiteExtInferiorPor.append(limInfExtPorc)
    LimiteExtSuperiorPorc.append(limSupExtPorc)
    df_outliers = pd.DataFrame(list(zip(Columnas,LimiteInferior, LimiteInferiorPor ,LimiteSuperior, LimiteSuperiorPorc, LimiteExInferior , LimiteExtInferiorPor, LimiteExtSuperior, LimiteExtSuperiorPorc)), 
                             columns = ['Feature','Valor límite inferior','Valor límite inferior %', 'Valor límite superior','Valor límite superior %', 'Valor extremo inferior','Valor extremo inferior %',
                                       'Valor extremo superior','Valor extremo superior %'])

In [ ]:
df_outliers

No se observan valores outliers con representación relevante

## 1.4 Distribución de las features

In [ ]:
# Distribuciones de las features
features_numericas.hist(bins =20 ,figsize=(20,15))
plt.show()

In [ ]:
# Boxplots
# Datos numéricos
a = 4 # numero filas
b = 4  # numero columnas
c = 1  # contador plots

fig = plt.figure(figsize=(24,16))

for i in features_numericas:
    sns.set(style="darkgrid")
    plt.subplot(a, b, c)
    plt.title('{}'.format(i))
    plt.xlabel(i)
    sns.boxplot(y=features_numericas[i],palette="Blues", orient="v")
    c = c + 1

plt.show()

## 1.5 Pairplot

In [ ]:
sns.pairplot(features_numericas, diag_kind="kde")

## 1.6 Comprobación de la normalidad de las features

In [ ]:
# Comproba la normalidad del conjunto de features numéricas, se aplica el test estadísitico Shapiro Wilk
#from scipy.stats import shapiro

alpha = 0.05
Variable = []
Est = []
Pval = []
Resul = []
for i in features_numericas:
    #print ("\nFeature" ,[i])
    a,b = stats.shapiro(features_numericas[[i]])
    #print( "Estadístico", a , "p-valor" , b)
    if b < alpha:
        
        Variable.append(i)
        Est.append(a)
        Pval.append(b)
        Resul.append("Se rechaza la hipótesis nula, la muestra no proviene de una distribución normal")
    else:
        Variable.append([i])
        Est.append(a)
        Pval.append(b)
        Resul.append("Se acepta la hipótesis nula, la muestra proviene de una distribución normal")
        
    df_Normalidad = pd.DataFrame(list(zip(Variable,Est, Pval ,Resul)), 
                             columns = ["Fature", "Estadístico", "P-valor", "Resultado"])

In [ ]:
pd.set_option('max_colwidth', 800)
df_Normalidad[:21]

## 1.7 Comprobación de correlación entre las features

In [ ]:
# Matriz de correlación
## Corresponde aplicar spearman pues segun el test de normalidad las features (con la excepcion de Ash_Alcanity) no son normales.
corr_matrix = dataset.iloc[:, 0:13].corr(method='spearman')
corr_matrix = round(corr_matrix ,3)
corr_matrix

In [ ]:
fig = plt.figure(figsize=(24,10))
sns.heatmap(corr_matrix, 
            annot=True,
            cbar_kws={"label": "Correlation coefficient", "shrink": 1.25, "ticks": np.linspace(-1.0, 1.0, 9)}, 
            cmap=plt.cm.hot_r, 
            vmin=-1, vmax=1)
plt.show()

In [ ]:
# Función correlación
def fun_corr_matrix(corr_mat):

    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)
    
    return(corr_mat)

In [ ]:
# Tabla correlción
corr_matrix = dataset.iloc[:, 0:13].corr(method='spearman')
fun_corr_matrix(corr_matrix).head(10)

Rangos de correlación propuestos:

- Correlación < 0,3: correlación débil.
- Correlación entre 0,3 y 0,7: correlación moderada
- Correlación > 0,7: correlación fuerte

Se obervan fuertes correlaciones por lo que deberíamos el valorar la posibilidad de descartar algunas variables.

***

# 2. Modelo de clasificación

In [ ]:
# Dividir el data set en conjunto de entrenamiento y conjunto de testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Escalado de variables
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)